### Package needed!

In [ ]:
#pip install flask

In [ ]:
import json, time, ast, queue, socket, sqlite3, datetime, threading, random, pandas
import time 

convert_timestamp = lambda timestamp: datetime.datetime.fromtimestamp(timestamp+28800).strftime('%Y-%m-%d %H:%M') 
# initialize database 
def initialize_database(): 
    con = sqlite3.connect("webpages.db")  
    cur = con.cursor()  
    cur.execute("CREATE TABLE IF NOT EXISTS webpage (url TEXT, content BLOB, timestamp TIMESTAMP DEFAULT (strftime('%Y-%m-%dT%H:%M:%S','now', 'localtime')));")
    # cur.execute('INSERT INTO webpage(url, content) VALUES (?,?);', ('1',b'2')) 
    con.commit() 
    con.close()
    
initialize_database()

In [ ]:
import queue, sqlite3, threading
class StorageDB():
    def __init__(self, db="webpages.db"):
        self.db = db # database file path
        self.q_in  = queue.Queue()
        self.q_out = queue.Queue() 
        threading.Thread(target=self.__database, name='thread-1', daemon=True).start()
        
    def __database(self): 
        con   = sqlite3.connect(self.db)
        cur   = con.cursor() 
        while True: 
            opt, foo = self.q_in.get() 
            bar = False 
            try:
                if   opt == 'new_html':
                    cur.execute('INSERT INTO webpage(url, content) VALUES (?,?);', foo) 
                    bar = True
                elif opt == 'commit':
                    con.commit()
                    bar = 'Database commit'
                    print('Database commit')
                elif opt == 'close':
                    con.close()
                    bar = 'Database close' 
                    print('Database close')
                elif opt == 'execute':  
                    cur.execute(foo)
                    bar = cur.fetchall()  
                else:
                    print('sql_else')
            except Exception as e:
                print('sql_exception', e, opt, foo)
                pass
            self.q_out.put(bar) 
            self.q_out.join()
            self.q_in.task_done()
            if opt == 'close': break 

    def operator(self, opt_foo): 
        #db.operator(('new_html',('cccccccc',b'oko31313')))
        #db.operator(('execute', "select * from webpage"))
        #db.operator(('commit',None))
        self.q_in.join()        
        self.q_in.put(opt_foo)  
        bar = self.q_out.get()  
        self.q_out.task_done()   
        return bar  

db = StorageDB()

### Set up a web server

In [ ]:
from flask import Flask, request, make_response, jsonify
import flask
app = Flask(__name__)  

#### URL provider

In [ ]:
@app.route('/urlProvider', methods = ['POST']) 
def urlProvider():
    url = [
        'https://www.wikipedia.org/'
    ]
    additional_url = [i.strip() for i in request.data.decode().splitlines()]
    url.extend(additional_url)
    response = jsonify(url=url)
    response.headers['Access-Control-Allow-Origin'] = '*'
    print('[URL to scrape]')
    print('\n'.join(url),'\n')
    return response

#### Webpage Store

In [ ]:
@app.route('/webpageStore', methods = ['POST']) 
def webpageStore():
    url = request.form['url'] 
    doc = request.form['doc'].encode()
    db.operator(('new_html', (url, doc)))
    print('[New HTML in DB]', url)
    return 'OK'

In [ ]:
@app.route('/commit', methods = ['GET']) 
def commit():
    db.operator(('commit',None))
    return 'OK'

### Run both apps

In [ ]:
app.run('0.0.0.0', 9999)

### Inspect the result

In [ ]:
df = pandas.read_sql_query("SELECT * FROM webpage", sqlite3.connect('webpages.db'))
df

In [ ]:
from IPython.core.display import HTML
display(HTML(df.loc[0,'content'].decode()))

#### Extract titles from HTMLs

In [ ]:
import re
title = re.compile(b"<title(?![0-9A-Za-z]).*?>.*?</title>", re.DOTALL)

for row in df.itertuples():  
    m = re.search(title, row.content) 
    print(m.group(0))